In [1]:
#librerias para analizar datos
import os
import numpy as np
import pandas as pd
from datetime import datetime
#librerias de graficos
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table, select

pd.set_option('display.max_colwidth', None)
pd.set_option('float_format', '{:,.2f}'.format)

In [2]:
products = pd.read_csv('data/Products.txt', delimiter='|', names=['product_id', 'detalle', 'tipoEnvase'])
products.head(2)

,product_id,detalle,tipoEnvase
0,1,Kool Cola,1 Liter
1,2,Kool Cola,2 Liter


In [3]:
#Transformacion de datos de los productos
df_products = products.copy() #copiamos los datos para no trabajar directamente con los originales
df_products['product_id'] = df_products['product_id'].astype(str).str.rstrip().astype(int)
df_products['detalle'] = df_products['detalle'].str.rstrip()
df_products.head(5)

,product_id,detalle,tipoEnvase
0,1,Kool Cola,1 Liter
1,2,Kool Cola,2 Liter
2,3,Kool Cola,500 cm3 can
3,4,Diet Cola,1 Liter
4,5,Diet Cola,2 Liter


In [4]:
df_products['old_package'] = df_products['tipoEnvase']
df_products['tipoEnvase'] = np.where(df_products['tipoEnvase'].str.contains('can')
                                  , 'Can'
                                  , 'Bottle')
df_products.head(2)

,product_id,detalle,tipoEnvase,old_package
0,1,Kool Cola,Bottle,1 Liter
1,2,Kool Cola,Bottle,2 Liter


In [5]:
#creamos una funcion para calcular los litros que tiene cada producto
def transform_liters(list):
    if list[1] == 'Liter':
        return float(list[0])
    elif list[1] == 'cm3':
        return float(list[0])/1000
    else:
        np.nan

#y ahora la utilizamos
df_products['splitted'] = df_products['old_package'].str.split(' ')
df_products['litros'] = df_products['splitted'].apply(lambda x: transform_liters(x))
df_products.head(2)

,product_id,detalle,tipoEnvase,old_package,splitted,litros
0,1,Kool Cola,Bottle,1 Liter,"[1, Liter]",1.00
1,2,Kool Cola,Bottle,2 Liter,"[2, Liter]",2.00


In [6]:
#ya no necesitamos la columnas que creamos recien llamadas old_package y splitted, entonces la borramos
df_products = df_products.drop(columns = ['old_package', 'splitted'])
df_products.head(20)

#Fin Transformacion de datos de los productos

,product_id,detalle,tipoEnvase,litros
0,1,Kool Cola,Bottle,1.00
1,2,Kool Cola,Bottle,2.00
2,3,Kool Cola,Can,0.50
3,4,Diet Cola,Bottle,1.00
4,5,Diet Cola,Bottle,2.00
5,6,Diet Cola,Can,0.50
6,7,Caffeine Free Cola,Bottle,1.00
7,8,Caffeine Free Cola,Bottle,2.00
8,9,Caffeine Free Cola,Can,0.50
9,10,Old Fashioned Root Beer,Bottle,1.00


In [7]:
# Función para clasificar el tipo de bebida
def clasificar_bebida(detalle):
    detalle = detalle.lower()
    if 'diet' in detalle:
        return 'Bebida de dieta'
    elif 'caffeine' in detalle:
        return 'Bebida de cafeína'
    elif 'energy' in detalle:
        return 'Bebida energética'
    elif 'kool' in detalle:
        return 'Bebida Kool'
    elif 'root' in detalle:
        return 'Bebida Root'
    elif 'juice' in detalle:
        return 'Jugo'
    elif 'soda' in detalle:
        return 'Bebida de soda'
    else:
        return 'Otro tipo de bebida'

# Aplicar la función al DataFrame
df_products['tipoBebida'] = df_products['detalle'].apply(clasificar_bebida)

df_products.head(2)



,product_id,detalle,tipoEnvase,litros,tipoBebida
0,1,Kool Cola,Bottle,1.00,Bebida Kool
1,2,Kool Cola,Bottle,2.00,Bebida Kool


In [8]:
zonaGeografica = pd.read_csv('data/Regions.txt', delimiter = '|', names = ['region','estado','ciudad','id'])
zonaGeografica.head(2)


,region,estado,ciudad,id
0,West,California,San Jose,91520
1,West,California,Morgan Hill,95200


In [9]:
df_zona = zonaGeografica.copy()
df_zona.head(5)

,region,estado,ciudad,id
0,West,California,San Jose,91520
1,West,California,Morgan Hill,95200
2,West,California,Gilroy,95300
3,East,New York,Manhattan,12111
4,East,New York,Manhattan,12112


In [10]:
load_dotenv()
DATABASE_URL = f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(DATABASE_URL)

metadata = MetaData()
billing = Table('billing', metadata, autoload_with=engine)
billing_detail = Table('billing_detail', metadata, autoload_with=engine)
prices = Table('prices', metadata, autoload_with=engine, schema='sales')

def query_sales_after_2008():
    with engine.connect() as connection:
        # Usar el constructor de consultas para generar la consulta
        query = select(
            billing.c.BILLING_ID.label('billing_id'),
            billing.c.REGION.label('region'),
            billing.c.BRANCH_ID.label('branch_id'),
            billing.c.DATE.label('date'),
            billing.c.CUSTOMER_ID.label('customer_id'),
            billing.c.EMPLOYEE_ID.label('employee_id'),
            billing_detail.c.PRODUCT_ID.label('product_id'),
            billing_detail.c.QUANTITY.label('quantity')
        ).select_from(
            billing.join(billing_detail, billing.c.BILLING_ID == billing_detail.c.BILLING_ID)
        )
        df = pd.read_sql(query, connection)
        return df

def query_sales_prices():
    with engine.connect() as connection:
        # Construir la consulta utilizando el constructor de consultas
        query = select(
            billing.c.BILLING_ID.label('billing_id'),
            billing.c.REGION.label('region'),
            billing.c.BRANCH_ID.label('branch_id'),
            billing.c.DATE.label('date'),
            billing.c.CUSTOMER_ID.label('customer_id'),
            billing.c.EMPLOYEE_ID.label('employee_id'),
            billing_detail.c.PRODUCT_ID.label('product_id'),
            billing_detail.c.QUANTITY.label('quantity'),
            prices.c.DATE.label('date_price'),
            prices.c.PRICE.label('price')
        ).select_from(
            billing.join(billing_detail, billing.c.BILLING_ID == billing_detail.c.BILLING_ID)
                  .join(prices, billing_detail.c.PRODUCT_ID == prices.c.PRODUCT_ID)
        )

        # Ejecutar la consulta y cargar los resultados en un DataFrame de pandas
        df = pd.read_sql(query, connection)
        return df
    
history_sales_after_2008 = query_sales_after_2008()
history_sales_after_2008.head(2)

,billing_id,region,branch_id,date,customer_id,employee_id,product_id,quantity
0,835920,South,1,2009-01-01 00:03:48,2298,242,2,5
1,835920,South,1,2009-01-01 00:03:48,2298,242,5,11


In [11]:
#copio los datos originales asi no los perjudico
df_sales = history_sales_after_2008.copy()
#convierto la columna product_id a integer
df_sales['product_id'] = df_sales['product_id'].fillna(-1).astype(int)
#mergeo con productos para obtener los litros de cada producto y asi obtener los litros vendidos en cada fila
df_sales = df_sales.merge(df_products, how='inner', on='product_id')
df_sales.head(2)

,billing_id,region,branch_id,date,customer_id,employee_id,product_id,quantity,detalle,tipoEnvase,litros,tipoBebida
0,835920,South,1,2009-01-01 00:03:48,2298,242,2,5,Kool Cola,Bottle,2.00,Bebida Kool
1,835920,South,1,2009-01-01 00:03:48,2298,242,5,11,Diet Cola,Bottle,2.00,Bebida de dieta


In [23]:
df_products_definitive = df_sales.copy()
df_products_definitive = df_products_definitive.drop(columns=['billing_id', 'customer_id', 'employee_id','branch_id'])
df_products_definitive.head(20)

,region,date,product_id,quantity,detalle,tipoEnvase,litros,tipoBebida
0,South,2009-01-01 00:03:48,2,5,Kool Cola,Bottle,2.00,Bebida Kool
1,South,2009-01-01 00:03:48,5,11,Diet Cola,Bottle,2.00,Bebida de dieta
2,South,2009-01-01 00:03:48,14,4,Diet Root Beer,Bottle,2.00,Bebida de dieta
3,South,2009-01-01 00:03:48,20,22,Birch Beer,Bottle,2.00,Otro tipo de bebida
4,South,2009-01-01 00:03:48,25,10,Vanilla Cream Soda,Bottle,1.00,Bebida de soda
5,South,2009-01-01 00:05:45,10,5,Old Fashioned Root Beer,Bottle,1.00,Bebida Root
6,South,2009-01-01 00:05:45,27,3,Vanilla Cream Soda,Bottle,0.67,Bebida de soda
7,South,2009-01-01 00:05:45,31,2,Diet Cream Soda,Can,0.50,Bebida de dieta
8,South,2009-01-01 00:05:45,42,10,Orval Beer,Can,0.33,Otro tipo de bebida
9,North,2009-01-01 00:05:45,5,14,Diet Cola,Bottle,2.00,Bebida de dieta


In [17]:
# Crear la nueva columna 'Sucursal'
#df_products_definitive['Sucursal'] = 'Sucursal nro ' + df_products_definitive['branch_id'].astype(str)
#df_products_definitive.head(2)

,region,branch_id,product_id,detalle,tipoEnvase,litros,tipoBebida,Sucursal
0,South,1,2,Kool Cola,Bottle,2.00,Bebida Kool,Sucursal nro 1
1,South,1,5,Diet Cola,Bottle,2.00,Bebida de dieta,Sucursal nro 1


In [24]:
df_products_definitive.to_csv('productos.csv', index=False)